<a href="https://colab.research.google.com/github/ajafarsadiq2002/Fake-News-Classifier-Using-Bidirectional-LSTM-RNN/blob/main/FakeNewsClassifierUsingBidirectionalLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**HOW TO IMPORT DATA SETS FROM KAGGLE ,STEPS TO BE FOLLOWED ARE GIVEN DOWN**

In [1]:
!pip install kaggle

In [2]:

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ajafarsadiq2002","key":"516e140961b3d4b26dd1d635bd82f0ef"}'}

In [3]:
!mkdir ~/.kaggle

In [4]:
!mv kaggle.json ~/.kaggle/


In [5]:
!chmod 600 ~/.kaggle/kaggle.json


In [6]:
! kaggle datasets list


ref                                                       title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
hummaamqaasim/jobs-in-data                                Jobs and Salaries in Data Science                   76KB  2023-12-25 20:03:32           8334        171  1.0              
sazidthe1/data-science-salaries                           Data Science Salaries 2024                          57KB  2024-01-20 16:45:27            933         29  1.0              
nelgiriyewithana/apple-quality                            Apple Quality                                      170KB  2024-01-11 14:31:07           2330         60  1.0              
huzdaria/laptop-pricing                                   Laptop Pricing                       

In [7]:
! kaggle competitions download -c fake-news


 82% 38.0M/46.5M [00:00<00:00, 89.0MB/s]
100% 46.5M/46.5M [00:00<00:00, 90.1MB/s]


In [8]:
!unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


**Fake News Classifier Using Bidirectional RNN**

In [9]:
import pandas as pd


In [10]:
df=pd.read_csv('train.csv')


In [11]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
###Drop Nan Values
df=df.dropna()

In [13]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [14]:
## Get the Dependent features
y=df['label']

In [15]:
y.value_counts()



0    10361
1     7924
Name: label, dtype: int64

In [16]:
X.shape


(18285, 4)

In [17]:
y.shape


(18285,)

In [18]:
import tensorflow as tf


In [19]:
tf.__version__


'2.15.0'

In [20]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [21]:
### Vocabulary size
voc_size=5000

**Onehot Representation**

In [22]:
messages=X.copy()


In [23]:
messages['title'][1]


'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [24]:
messages.reset_index(inplace=True)


In [25]:
import nltk
import re
from nltk.corpus import stopwords

In [26]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [28]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [29]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[3225, 2483, 4538, 3181, 3020, 4881, 2916, 3646, 714, 3640],
 [1284, 1217, 1200, 1986, 1006, 379, 4560],
 [4479, 2541, 2105, 2645],
 [4687, 177, 606, 3061, 534, 3917],
 [4752, 1006, 3515, 1186, 292, 3385, 1006, 1244, 4887, 4741],
 [3885,
  700,
  3284,
  1673,
  3382,
  1871,
  4182,
  470,
  301,
  341,
  668,
  2305,
  1849,
  3728,
  4560],
 [2000, 2986, 4199, 2960, 727, 606, 3211, 2142, 664, 2491, 4417],
 [4937, 1303, 125, 4600, 4823, 4660, 1871, 4263, 664, 2491, 4417],
 [4163, 446, 4879, 4216, 517, 687, 102, 2191, 1871, 2255],
 [1150, 3776, 1811, 1217, 531, 3361, 4044, 1636],
 [2678, 1138, 3951, 3823, 1721, 4644, 2446, 3754, 934, 3651, 2094],
 [3061, 4956, 3020, 687, 1871, 4823],
 [4398, 3558, 4558, 803, 539, 4611, 4449, 2558, 2439],
 [3266, 1902, 1635, 1495, 2549, 3778, 4879, 664, 2491, 4417],
 [1540, 127, 727, 4917, 3416, 664, 2491, 4417],
 [4111, 445, 497, 1, 1384, 1758, 4988, 4157, 2685, 189],
 [1255, 1633, 1217],
 [1602, 776, 4764, 1379, 1871, 3008, 93, 4560],
 [1182, 1265, 

**Embedding Representation**

In [30]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3646  714 3640]
 [   0    0    0 ... 1006  379 4560]
 [   0    0    0 ... 2541 2105 2645]
 ...
 [   0    0    0 ...  664 2491 4417]
 [   0    0    0 ... 4646 4171 3756]
 [   0    0    0 ... 4082 3876 2617]]


In [31]:

embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3225,
       2483, 4538, 3181, 3020, 4881, 2916, 3646,  714, 3640], dtype=int32)

In [32]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [33]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [34]:
len(embedded_docs),y.shape


(18285, (18285,))

In [35]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [36]:
X_final.shape,y_final.shape


((18285, 20), (18285,))

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


**Model Training**

In [38]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 17s 64ms/step - loss: 0.3027 - accuracy: 0.8533 - val_loss: 0.1968 - val_accuracy: 0.9173
Epoch 2/10
192/192 [==============================] - 12s 62ms/step - loss: 0.1323 - accuracy: 0.9517 - val_loss: 0.2009 - val_accuracy: 0.9178
Epoch 3/10
192/192 [==============================] - 12s 62ms/step - loss: 0.0913 - accuracy: 0.9667 - val_loss: 0.2131 - val_accuracy: 0.9163
Epoch 4/10
192/192 [==============================] - 12s 65ms/step - loss: 0.0624 - accuracy: 0.9793 - val_loss: 0.2639 - val_accuracy: 0.9150
Epoch 5/10
192/192 [==============================] - 12s 65ms/step - loss: 0.0382 - accuracy: 0.9868 - val_loss: 0.3272 - val_accuracy: 0.9142
Epoch 6/10
192/192 [==============================] - 12s 62ms/step - loss: 0.0191 - accuracy: 0.9940 - val_loss: 0.4537 - val_accuracy: 0.9087
Epoch 7/10
192/192 [==============================] - 11s 59ms/step - loss: 0.0146 - accuracy: 0.9962 - val_loss: 0.5000 - val_accuracy:

**Performance Metrics And Accuracy**

In [40]:
y_pred1=(model1.predict(X_test) > 0.5)


189/189 [==============================] - 3s 7ms/step


In [41]:
from sklearn.metrics import confusion_matrix


In [42]:
confusion_matrix(y_test,y_pred1)


array([[3123,  296],
       [ 265, 2351]])

In [43]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9070422535211268

In [44]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      3419
           1       0.89      0.90      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

